In [2]:
%load_ext autoreload
%autoreload 2

In [31]:
from tools.handlers import Zip2CSV
import pandas as pd
import spacy
import re
import json
from itertools import groupby

In [32]:

nlp = spacy.load("en_core_web_sm")

In [37]:
corpus = Zip2CSV(nlp='0002247', directory='data')
corpus.convert()

0it [00:00, ?it/s]

In [38]:
!ls -la data 

total 483376
drwxr-xr-x  10 kasparbeelen  staff        320 Jun  5 14:04 .
drwxr-xr-x@ 10 kasparbeelen  staff        320 Jun  1 16:10 ..
-rw-r--r--@  1 kasparbeelen  staff       6148 Jun  5 14:04 .DS_Store
drwxr-xr-x@ 12 kasparbeelen  staff        384 May 31 14:47 0002247
-rw-r--r--@  1 kasparbeelen  staff  194532419 Jun  5 14:38 0002247.csv
drwxr-xr-x   4 kasparbeelen  staff        128 May 31 14:51 metadata
drwxr-xr-x   5 kasparbeelen  staff        160 Jun  5 14:25 ner-tagging
drwxr-xr-x   5 kasparbeelen  staff        160 Jun  5 14:35 plaintext
-rw-r--r--@  1 kasparbeelen  staff   46130332 Jun  1 10:21 predictions.json
-rw-r--r--@  1 kasparbeelen  staff        575 Jun  1 12:15 test.txt


In [39]:
df = pd.read_csv('data/0002247.csv',index_col=0)
df.head()

,title,item_type,ocr_quality_mean,date,content
0002247/1869/0320/0002247_18690320_art0001_metadata.xml,NaN,ARTICLE,0.8326,1869-03-20,new EDIN BORO\nthe fa3hionable Oliva an 4 Brow...
0002247/1869/0320/0002247_18690320_art0023_metadata.xml,SINGULA_R DisT'URBANCES.,ARTICLE,0.9487,1869-03-20,SINGULA_R DisT'URBANCES.\n\nFor upwards of 12 ...
0002247/1869/0320/0002247_18690320_art0065_metadata.xml,BUILDERS' TENDERS.,ARTICLE,0.9288,1869-03-20,BUILDERS' TENDERS.\n\nFor residence near Kette...
0002247/1869/0320/0002247_18690320_art0005_metadata.xml,PARLIAMENTARY SUMMARY,ARTICLE,0.9459,1869-03-20,PARLIAMENTARY SUMMARY\n\n---: 0 :-\n\nIN THE H...
0002247/1869/0320/0002247_18690320_art0037_metadata.xml,GENERAL NEWS.,ARTICLE,0.9480,1869-03-20,GENERAL NEWS.\n\nWaEcKs.—During the past week ...


In [90]:

vocab_df = pd.read_csv('./data/Railways_Webscrape_v1.csv')
vocab = vocab_df.Title.str.lower().values

In [97]:
query_vocab =  '|'.join([fr'\b{q.strip()}\b' for q in vocab])
regex_vocab =  re.compile(fr"({query_vocab})",re.I)

In [98]:
query_vocab

"\\b1,500v dc locomotive\\b|\\b3-hole disc wheel\\b|\\b3-link coupling\\b|\\b25kv ac locomotive\\b|\\b45 degree tappet lever frame\\b|\\b630v dc locomotive\\b|\\b660v dc locomotive\\b|\\babcl\\b|\\babsence switch\\b|\\babt rack locomotive\\b|\\babutment\\b|\\baccess agreement\\b|\\baccess component\\b|\\b<access tools and equipment>\\b|\\baccess tower\\b|\\baccommodation bridge\\b|\\baccommodation level crossing\\b|\\baccompanied article ticket\\b|\\baccompanied dog ticket\\b|\\baccompanied luggage ticket\\b|\\b<accounting plates, etc.>\\b|\\b<accounting records and equipment>\\b|\\baccumulator (locomotive)\\b|\\bactive suspension\\b|\\badaptor vehicle\\b|\\badaptor wagon\\b|\\badhesion railway\\b|\\badjustment switch\\b|\\b<administration>\\b|\\badriatic locomotive\\b|\\badvanced starter\\b|\\badvert\\b|\\badvertisement\\b|\\b<advertising and publicity>\\b|\\badvertising poster\\b|\\badvertising sign\\b|\\badvice note\\b|\\baeroplane truck\\b|\\bagricultural implement wagon\\b|\\bagri

In [82]:

# s='Hello, this is a string'
# #m = re.search(,s)
# #m.group()
# #m.start()

# import re
# p = re.compile(r'\bthis\b|\bis\b')
# for m in p.finditer(s):
#     print(m.start(),m.end(), m.group())

7 11 this
12 14 is


In [99]:
def tokenized_text(text):
    return list([tok.text for tok in nlp.tokenizer(text)])

In [100]:
df['tokenized_text'] = df.content.apply(tokenized_text)

In [101]:
def regex_text_to_spans(tokens, regex_vocab):
    results = []
    char_count = 0
    text = ' '.join(tokens)
    query_results = text.finditer(regex_vocab)
    for r in query_results:
            results.append({
                'from_name': 'label',
                'to_name': 'text',
                'type': 'labels',
                'value': {
                    'start': r.start(),
                    'end': r.end(),
                    'text': r.group(),
                    'labels': ['Object']
                }
            })
    
    return results

In [103]:
df['spans'] = df.tokenized_text.apply(regex_text_to_spans, regex_vocab=regex_vocab)

NameError: name 'finditer' is not defined

In [69]:
def text_to_spans(tokens, target_vocab):
    results = []
    char_count = 0
    text = []
    for i,token in enumerate(tokens):
            text.append(token)
            char_count+=len(token) + 1
            if token.lower() not in target_vocab:
                continue

            results.append({
                'from_name': 'label',
                'to_name': 'text',
                'type': 'labels',
                'value': {
                    'start': char_count-len(token)-1,
                    'end': char_count-1,
                    'text': token,
                    'labels': ['Object']
                }
            })
    
    return results

In [70]:
#df['formatted_content'] = df.content.apply(lambda x: ' '.join(x.split()))

In [71]:
df['spans'] = df.tokenized_text.apply(text_to_spans, target_vocab={'train'})

In [72]:
tasks = []
for i,row in df[df.spans.apply(lambda x: len(x)) > 0].iterrows():

    tasks.append({
        'data': {'text': ' '.join(row.tokenized_text)},
        'predictions': [{'model_version': 'match', 'result': row.spans}]
    })

In [73]:
# Save Label Studio tasks.json
print(f'Save {len(tasks)} tasks to "tasks.json"')
with open('data/ner-tagging/tasks.json', mode='w') as f:
    json.dump(tasks, f, indent=2)
    
# Save class labels as a txt file
print('Named entities are saved to "named_entities.txt"')
with open('data/ner-tagging/named_entities.txt', mode='w') as f:
    f.write('Object')

Save 2508 tasks to "tasks.json"
Named entities are saved to "named_entities.txt"
